In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from utils import ours_read_csv
from sklearn.metrics import mean_squared_error


DIR_PATH_TOM = "C:\\Users\\ttomn\\OneDrive\\Desktop\\data-science-transportation-research"

In [2]:
file_name = "10000_streets_final_df.csv"

In [3]:
df = ours_read_csv(f"{DIR_PATH_TOM}\\{file_name}")
df

,ST_INDEX,the_geom,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,Social Services,...,Bridge,Tunnel,Boardwalk,PathTrail,StepStreet,Driveway,Ramp,Alley,U-Turn,FerryRoute
0,12,MULTILINESTRING ((-74.01213630770864 40.692040...,1,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
1,12,MULTILINESTRING ((-74.01213630770864 40.692040...,2,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,12,MULTILINESTRING ((-74.01213630770864 40.692040...,3,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,12,MULTILINESTRING ((-74.01213630770864 40.692040...,4,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,12,MULTILINESTRING ((-74.01213630770864 40.692040...,5,2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286663,121095,MULTILINESTRING ((-73.88110596307965 40.567784...,10,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1286664,121095,MULTILINESTRING ((-73.88110596307965 40.567784...,11,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1286665,121095,MULTILINESTRING ((-73.88110596307965 40.567784...,12,2022,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1286666,121095,MULTILINESTRING ((-73.88110596307965 40.567784...,1,2023,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
df = df.drop(columns=["the_geom", "ST_INDEX"])

KeyError: "['the_geom', 'ST_INDEX'] not found in axis"

In [6]:
y = df.astype(np.float64)['COLLISIONS']
X = df.astype(np.float64).drop('COLLISIONS', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)

In [7]:
model = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the model summary
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             COLLISIONS   No. Observations:               965001
Model:                            GLM   Df Residuals:                   964962
Model Family:        NegativeBinomial   Df Model:                           38
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -3.1158e+05
Date:                Sat, 03 Jun 2023   Deviance:                   3.9948e+05
Time:                        17:03:15   Pearson chi2:                 1.83e+06
No. Iterations:                    29   Pseudo R-squ. (CS):            0.06602
Covariance Type:            nonrobust                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
MONTH                     0.02

In [16]:
y_pred = result.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)/y_pred.shape[0]

In [15]:
pd.DataFrame({'true':y_test, 'pred':y_pred})

,true,pred
332638,0.0,0.169454
52947,0.0,0.223309
458438,1.0,0.124692
607804,0.0,0.047024
1238724,0.0,0.297516
...,...,...
320509,0.0,0.107088
235268,0.0,0.013101
1200818,0.0,0.096100
320366,0.0,0.135709


In [17]:
with open('output_10000.txt', 'w') as file:
    file.write(str(result.summary()) + '\n')
    file.write(f"normalized MSE on test data is {mse_test}")